In [1]:
import os
import librosa
import numpy as np
from scipy.ndimage import maximum_filter

def get_audio_files_from_dir(directory, extensions={".wav", ".mp3"}):
    return [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if os.path.splitext(f)[1].lower() in extensions
    ]

def get_spectrogram_peaks(y, sr, n_fft=4096, hop_length=512, threshold_db=-30):
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    S_db = librosa.amplitude_to_db(S, ref=np.max)
    
    # Local maxima detection
    neighborhood_size = (20, 20)
    local_max = (S_db == maximum_filter(S_db, footprint=np.ones(neighborhood_size)))
    peaks = np.argwhere(local_max & (S_db > threshold_db))  # (freq_bin, time_bin)

    return peaks

def generate_peak_pairs(peaks, fanout=5, time_window=50):
    pairs = []
    for i in range(len(peaks)):
        freq1, t1 = peaks[i]
        for j in range(1, fanout):
            if i + j < len(peaks):
                freq2, t2 = peaks[i + j]
                delta_t = t2 - t1
                if 0 < delta_t <= time_window:
                    pairs.append((freq1, freq2, delta_t, t1))
    return pairs

def process_audio_directory(directory_path):
    all_fingerprints = []
    audio_files = get_audio_files_from_dir(directory_path)

    for audio_path in audio_files:
        try:
            filename = os.path.basename(audio_path)
            y, sr = librosa.load(audio_path, sr=None, mono=True)
            peaks = get_spectrogram_peaks(y, sr)
            peak_pairs = generate_peak_pairs(peaks)

            for (freq1, freq2, delta_t, anchor_time) in peak_pairs:
                all_fingerprints.append((filename, int(freq1), int(freq2), int(delta_t), int(anchor_time)))
        except Exception as e:
            print(f"[ERROR] Skipping {audio_path}: {e}")

    return all_fingerprints

In [2]:
directory_path = "songs_mp3"  # Change this to your folder path if needed

print(f"Processing songs in: {directory_path}")
all_fingerprints = process_audio_directory(directory_path)

Processing songs in: songs_mp3


In [3]:
import pandas as pd

df = pd.DataFrame(
    all_fingerprints,
    columns=["filename", "freq1", "freq2", "delta_time", "anchor_time"]
)

# Preview results
df.head()

,filename,freq1,freq2,delta_time,anchor_time
0,Billie Eilish - BIRDS OF A FEATHER (Official M...,3,3,21,17784
1,Billie Eilish - BIRDS OF A FEATHER (Official M...,3,3,20,18210
2,Billie Eilish - BIRDS OF A FEATHER (Official M...,4,4,20,15066
3,Billie Eilish - BIRDS OF A FEATHER (Official M...,4,4,37,15903
4,Billie Eilish - BIRDS OF A FEATHER (Official M...,4,4,39,17654


In [6]:
df.to_csv("Flattened_Fingerprints.csv", index=False)
print("Saved to Flattened_Fingerprints.csv")

Saved to Flattened_Fingerprints.csv


In [ ]:
import pandas as pd

df = pd.DataFrame(all_fingerprints, columns=["filename", "freq1", "freq2", "delta_time", "anchor_time"])
df.to_csv("flattened_fingerprints.csv", index=False)